In [141]:
import pandas as pd
import numpy as np
import requests
import psycopg2
import json
import calendar
import time
import re
import kaggle

import boto3
import base64
from botocore.exceptions import ClientError

OSError: Could not find kaggle.json. Make sure it's located in /home/ubuntu/.kaggle. Or use the environment method.

In [3]:
def get_secret():

    secret_name = "CCC_DB"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    return(secret)

In [4]:
def get_key():

    secret_name = "CCC_API"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    return(secret)

In [8]:
api_key = json.loads(get_key())['CCC_API']

In [9]:
api_key

'109ce37ef6bd6f319468d5530e38feb1ae7793272768e5b280717832b7d3bcd0'

In [5]:
secret = json.loads(get_secret())

engine = psycopg2.connect(
    database=secret["dbname"],
    user=secret["username"],
    password=secret["password"],
    host=secret["host"],
    port=secret["port"]
)

In [8]:
cur = engine.cursor()

print(cur.execute("select * from information_schema.tables"))

None


In [12]:
ts = int(time.time())

In [13]:
ts

1564100254

In [20]:
df = pd.DataFrame(index=pd.date_range(start='7/25/2018', end='7/25/2019'))

In [21]:
df.head()

""
2018-07-25
2018-07-26
2018-07-27
2018-07-28
2018-07-29


In [25]:
df.insert(loc=0,column='UnixTime',value=0)

In [26]:
df.head()

,UnixTime
2018-07-25,0
2018-07-26,0
2018-07-27,0
2018-07-28,0
2018-07-29,0


In [33]:
str(df.index[0])

'2018-07-25 00:00:00'

In [38]:
df['UnixTime'][0] = calendar.timegm(time.strptime(str(df.index[0]), '%Y-%m-%d %H:%M:%S'))

In [42]:
df['UnixTime'] = calendar.timegm(time.strptime(str(df.index), '%Y-%m-%d %H:%M:%S'))

ValueError: time data "DatetimeIndex(['2018-07-25', '2018-07-26', '2018-07-27', '2018-07-28',\n               '2018-07-29', '2018-07-30', '2018-07-31', '2018-08-01',\n               '2018-08-02', '2018-08-03',\n               ...\n               '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19',\n               '2019-07-20', '2019-07-21', '2019-07-22', '2019-07-23',\n               '2019-07-24', '2019-07-25'],\n              dtype='datetime64[ns]', length=366, freq='D')" does not match format '%Y-%m-%d %H:%M:%S'

In [45]:
for i in df.index:
    df['UnixTime'][i] = calendar.timegm(time.strptime(str(i), '%Y-%m-%d %H:%M:%S'))

In [46]:
df.head()

,UnixTime
2018-07-25,1532476800
2018-07-26,1532563200
2018-07-27,1532649600
2018-07-28,1532736000
2018-07-29,1532822400


In [47]:
base_url = 'https://min-api.cryptocompare.com/data/dayAvg?fsym=BTC&tsym=USD&toTS='

In [55]:
query_url = base_url + str(ts) + api_key

In [133]:
query_url

'https://min-api.cryptocompare.com/data/dayAvg?fsym=BTC&tsym=USD&toTS=1564012800109ce37ef6bd6f319468d5530e38feb1ae7793272768e5b280717832b7d3bcd0'

In [93]:
btc_list = np.zeros(366)

In [98]:
btc_list[0] = float(4.5)

In [115]:
j = 0
for i in df['UnixTime']:
    query_url = base_url + str(i) + api_key
    resp = requests.get(query_url)
    bystr = str(resp.content)
    price = re.search(r'USD":(.+?),', bystr).group(1)
    print(price + "   " + str(i) + "   " + str(df.index[j]))
    btc_list[j] = float(price)
    j = j+1
    

9725.17   1532476800   2018-07-25 00:00:00
9725.17   1532563200   2018-07-26 00:00:00
9725.17   1532649600   2018-07-27 00:00:00
9725.17   1532736000   2018-07-28 00:00:00
9725.17   1532822400   2018-07-29 00:00:00
9725.17   1532908800   2018-07-30 00:00:00
9725.17   1532995200   2018-07-31 00:00:00
9725.17   1533081600   2018-08-01 00:00:00
9725.17   1533168000   2018-08-02 00:00:00
9725.17   1533254400   2018-08-03 00:00:00
9725.17   1533340800   2018-08-04 00:00:00
9725.17   1533427200   2018-08-05 00:00:00
9725.17   1533513600   2018-08-06 00:00:00
9725.17   1533600000   2018-08-07 00:00:00
9725.17   1533686400   2018-08-08 00:00:00
9725.17   1533772800   2018-08-09 00:00:00
9725.17   1533859200   2018-08-10 00:00:00
9726.95   1533945600   2018-08-11 00:00:00
9725.17   1534032000   2018-08-12 00:00:00
9725.17   1534118400   2018-08-13 00:00:00
9725.17   1534204800   2018-08-14 00:00:00
9725.17   1534291200   2018-08-15 00:00:00
9725.17   1534377600   2018-08-16 00:00:00
9726.95   1

In [56]:
resp = requests.get(query_url)

In [57]:
resp

<Response [200]>

In [59]:
resp.content

b'{"USD":9728.05,"ConversionType":{"type":"direct","conversionSymbol":""}}'

In [74]:
bystr = str(resp.content)

In [81]:
re.search(r'USD":(.+?),', bystr).group(1)

'9728.05'

In [134]:
url2 = 'https://min-api.cryptocompare.com/data/daily/market/close?date=2019-07-21&api_key='
print(url2)
resp2 = requests.get(url2)

https://min-api.cryptocompare.com/data/daily/market/close?date=2019-07-21&api_key=109ce37ef6bd6f319468d5530e38feb1ae7793272768e5b280717832b7d3bcd0


In [135]:
resp2.content

b'{"Response":"Error","Message":"Not authorized","HasWarning":false,"Type":53,"RateLimit":{},"Data":{},"Cooldown":0}'

In [136]:
resp2.content.data

AttributeError: 'bytes' object has no attribute 'data'

In [142]:
kaggle datasets download -d philmohun/cryptocurrency-financial-data

SyntaxError: invalid syntax (<ipython-input-142-e2afd635b6af>, line 1)